In [1]:
#from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "movie" : "Top Gun",
        "answer" : "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "answer":"👨‍👨‍👦🔫🍝"
    }
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Please give me three emojis that represent the movie {movie}."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give three emojies."),
        example_prompt,
        ("human", "Please give me three emojis that represent the movie {movie}."),
    ]
)

chain = final_prompt | chat

template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory()

input_movie_1 = "Harry Potter"
output_emojies_1 = str(chain.invoke({"movie": input_movie_1}))
# will be saved to the memory

memory.save_context({"input": input_movie_1}, {"output": output_emojies_1})

input_movie_2 = "Moana"
output_emojies_2 = str(chain.invoke({"movie": input_movie_2}))
# will be saved to the memory

memory.save_context({"input": input_movie_2}, {"output": output_emojies_2})

memory.load_memory_variables({})


⚡🧙‍♂️🔮🌊🌺🚣‍♀️

{'history': "Human: Harry Potter\nAI: content='⚡🧙\\u200d♂️🔮'\nHuman: Moana\nAI: content='🌊🌺🚣\\u200d♀️'"}

In [2]:

from langchain.chains import ConversationChain


llm = ChatOpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory
)

conversation.predict(input="Harry Potter")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Harry Potter
AI: content='⚡🧙\u200d♂️🔮'
Human: Moana
AI: content='🌊🌺🚣\u200d♀️'
Human: Harry Potter
AI:

> Finished chain.


"content='⚡🧙\\u200d♂️🔮'"